In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.0 MB/s eta 0:00:00


In [ ]:
from tabulate import tabulate
from nltk.translate.bleu_score import sentence_bleu
from tqdm.auto import tqdm
import torch
import os
import random

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import string
import random
from tqdm.auto import tqdm
import torch
import transformers
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from transformers import AdamW, get_cosine_schedule_with_warmup
from transformers import pipeline, set_seed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
folder_path = '/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning'
for filename in os.listdir(folder_path):
    print(filename)

Finetuned_GPT2_10_epochs_V0.pt
FineTune_gpt2_engine_type_5_epochs.zip
FineTune_gpt2_drive_type_5_epochs.zip
FineTune_gpt2_drive_type_And_Engine_type_5_epochs.zip
FineTune_gpt2_question_answering_5_epochs.zip


In [ ]:
import torch
# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_drive_type_And_Engine_type_5_epochs.zip" -d FineTune_gpt2_drive_type_And_Engine_type_5_epochs
Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_drive_type_And_Engine_type_5_epochs')
Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs.to(device)

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_drive_type_And_Engine_type_5_epochs.zip
  inflating: FineTune_gpt2_drive_type_And_Engine_type_5_epochs/config.json  
  inflating: FineTune_gpt2_drive_type_And_Engine_type_5_epochs/pytorch_model.bin  
  inflating: FineTune_gpt2_drive_type_And_Engine_type_5_epochs/generation_config.json  


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
import logging
# Set the logging level to ERROR for the transformers library
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
def generate_prompt_zero_shot_1(row):
    prompt = f"Car: {row['Full Name']}\ndescription:"
    return prompt

In [ ]:
def generate_prompt_zero_shot_2(row):
    prompt = f"{row['Full Name']}"
    return prompt

In [ ]:
def generate_prompt_Few_shot(row, data):
    prompt1 = """Car: lexus rc-300 2022
    Description:  The lexus rc-300 2022 is a vehicle with a Inline 4 engine that delivers 241 horsepower and 258 lb-ft torque at 1,650. It has rear wheel drive  and a 8-speedshiftableautomatic. The car's dimensions are 185.0 in. in length, 54.9 in. in height, and 72.4 in width . Additionally, the base MSRP of this car is $43,895
    """
    prompt2 = """Car: bmw x5 2008
    Description:  The bmw x5 2008 is a vehicle with a Inline 6 engine that delivers 260 horsepower and 225 lb-ft torque at 2,750. It has all wheel drive  and a 6-speedshiftableautomatic. The car's dimensions are 191.1 in. in length, 69.5 in. in height, and 76.1 in width . Additionally, the base MSRP of this car is $46,200."""
    prompt3 = f"Car: {row['Full Name']}\ndescription:"
    prompt = f"{prompt1}\n{prompt2}\n{prompt3}"
    return prompt

In [ ]:
def calculate_bleu(model, tokenizer, data, prompt_fn, data_train):
    bleu_scores = []
    for index, row in tqdm(data.iterrows(), total=len(data), desc="Calculating BLEU score"):
        # Get the reference text from the 'prompt_target' column
        reference_text = row['prompt_target']
        
        # Generate the prompt using the provided function
        if prompt_fn.__name__ == 'generate_prompt_Few_shot':
            prompt = prompt_fn(row, data_train)
        else:
            prompt = prompt_fn(row)
        
        generated_text = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to(device)
        
        # Generate the text using the model
        output_text = model.generate(
            generated_text,
            max_length=250,
            temperature=0.005,
            do_sample=True,
            top_k=100,
            top_p=0.95,
            repetition_penalty=1.2,
            num_return_sequences=1,
        )
        decoded_output = tokenizer.decode(output_text[0], skip_special_tokens=True)
        # Calculate the BLEU score
        score = sentence_bleu([reference_text], decoded_output)
        bleu_scores.append(score)
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu_score

In [ ]:
def evaluate_prompt_functions(model, tokenizer, data, prompt_fns,data_train):
    results = []
    for prompt_fn_name, prompt_fn in tqdm(prompt_fns.items(), desc="Evaluating prompt functions"):
        avg_bleu_score = calculate_bleu(model, tokenizer, data, prompt_fn,data_train)
        results.append((prompt_fn_name, avg_bleu_score))

    # Print the results in a table format
    headers = ['Prompt Function', 'BLEU Score']
    table = tabulate(results, headers=headers, tablefmt='orgtbl')
    print(table)

In [ ]:
prompt_functions = { 'prompt_zero_shot_1' : generate_prompt_zero_shot_1,
                    'prompt_zero_shot_2' : generate_prompt_zero_shot_2,
                    'prompt_Few_shot' : generate_prompt_Few_shot

}

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Data/Summary_Final_context.csv")

In [ ]:
data= data.drop(columns=['Pros','Cons','horsepower','engine_type','Torque','Torque_RPM','drive_type','transmission','width','Length','Height'])

In [ ]:
prompts = []
targets = []
li = []
for index, row in data.iterrows():
    carName = str(row['Full Name']).replace('\n', ' ')
    description = str(row['final_context']).replace('\n', ' ')
    prompt = 'Car: ' + carName+'\nDescription: '
    target = description
    targets.append(target)
    prompts.append(prompt)
    li.append(prompt + ' ' + target)
data['prompt_target'] = li

In [ ]:
def divide_data(data):
    data = data.sample(frac=1, random_state=42)
    subset_size = len(data) // 2
    subset1 = data.iloc[:subset_size]
    subset2 = data.iloc[subset_size:]
    intersection = pd.merge(subset1, subset2, how='inner')
    if not intersection.empty:
        subset2 = subset2[~subset2.isin(intersection)].dropna()
    return subset1, subset2

In [ ]:
subset_train, subset_test = divide_data(data)

In [ ]:
subset_data = subset_test.sample(frac=0.25, random_state=42)

In [ ]:
subset_train_data = subset_test.sample(frac=0.25, random_state=42)

In [ ]:
subset_data

,base msrp,Full Name,GPT-2 Summarization,Context,final_context,prompt_target
3067,"$33,255",honda cr-v-hybrid 2022,The 2022 Honda CR-V Hybrid is part of the gro...,The honda cr-v-hybrid 2022 is a vehicle with a...,The honda cr-v-hybrid 2022 is a vehicle with a...,Car: honda cr-v-hybrid 2022\nDescription: The...
1318,"$34,900",chevrolet express 2019,What the minivan is to the local youth soccer...,The chevrolet express 2019 is a vehicle with a...,The chevrolet express 2019 is a vehicle with a...,Car: chevrolet express 2019\nDescription: The...
1852,"$46,200",bmw x5 2008,"After a complete redesign last year, the BMW X...",The bmw x5 2008 is a vehicle with a Inline 6 e...,The bmw x5 2008 is a vehicle with a Inline 6 e...,Car: bmw x5 2008\nDescription: The bmw x5 200...
1438,"$32,925",ford explorer 2021,Ford redesigned its Explorer last year and is...,The ford explorer 2021 is a vehicle with a Inl...,The ford explorer 2021 is a vehicle with a Inl...,Car: ford explorer 2021\nDescription: The for...
1223,"$35,400",bmw x1 2021,The BMW X1 is a new-school crossover SUV with...,The bmw x1 2021 is a vehicle with a Inline 4 e...,The bmw x1 2021 is a vehicle with a Inline 4 e...,Car: bmw x1 2021\nDescription: The bmw x1 202...
...,...,...,...,...,...,...
2809,"$22,795",subaru forester 2018,A roomy cabin and good fuel economy make the ...,The subaru forester 2018 is a vehicle with a F...,The subaru forester 2018 is a vehicle with a F...,Car: subaru forester 2018\nDescription: The s...
815,not available,chevrolet blazer 2000,"Base models are dropped. The engine, exhaust a...",The chevrolet blazer 2000 is a vehicle with a ...,The chevrolet blazer 2000 is a vehicle with a ...,Car: chevrolet blazer 2000\nDescription: The ...
454,"$34,750",chevrolet tahoe 2004,Chevrolet offers revised 16- and 17-inch alumi...,The chevrolet tahoe 2004 is a vehicle with a V...,The chevrolet tahoe 2004 is a vehicle with a V...,Car: chevrolet tahoe 2004\nDescription: The c...
1104,"$60,700",bmw x5 2019,When the BMW X5 came out for the 2000 model y...,The bmw x5 2019 is a vehicle with a Inline 6 e...,The bmw x5 2019 is a vehicle with a Inline 6 e...,Car: bmw x5 2019\nDescription: The bmw x5 201...


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
evaluate_prompt_functions(Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs, tokenizer, subset_data,prompt_functions,subset_train_data)

Evaluating prompt functions:   0%|          | 0/3 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/393 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/393 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/393 [00:00<?, ?it/s]

| Prompt Function    |   BLEU Score |
|--------------------+--------------|
| prompt_zero_shot_1 |     0.320295 |
| prompt_zero_shot_2 |     0.326389 |
| prompt_Few_shot    |     0.206956 |
